In [1]:
from aocd import get_data

# Day 9: Disk Fragmenter

## Part One

In [60]:
import numpy as np

example_input = "2333133121414131402"
counts = np.array(list(example_input), dtype=int); counts

array([2, 3, 3, 3, 1, 3, 3, 1, 2, 1, 4, 1, 4, 1, 3, 1, 4, 0, 2])

In [61]:
len(counts)

19

First we need to convert the input into the file representation:

In [62]:
file_values = np.arange(0, len(list(example_input))//2 + 1)
empty_values = np.full(len(list(example_input))//2 + 1, '.')

In [63]:
import itertools
interleaved = np.array(list(itertools.chain(*zip(file_values, empty_values))), dtype=str); interleaved

array(['0', '.', '1', '.', '2', '.', '3', '.', '4', '.', '5', '.', '6',
       '.', '7', '.', '8', '.', '9', '.'], dtype='<U21')

In [64]:
blocks = np.repeat(interleaved[:len(counts)], counts); blocks

array(['0', '0', '.', '.', '.', '1', '1', '1', '.', '.', '.', '2', '.',
       '.', '.', '3', '3', '3', '.', '4', '4', '.', '5', '5', '5', '5',
       '.', '6', '6', '6', '6', '.', '7', '7', '7', '.', '8', '8', '8',
       '8', '9', '9'], dtype='<U21')

In [65]:
''.join(blocks)

'00...111...2...333.44.5555.6666.777.888899'

#### Apprach 1:

Now we have our file representation we need to actually do the work of defragging

In [15]:
def shift_blocks(arr):
    while '.' in arr:
        value = arr[-1]  # get last element 
        arr = np.delete(arr, -1)  # remove last element, numpy doesn't have pop()
        
        dot_index = np.where(arr == '.')[0][0]
        arr[dot_index] = value
    
    return arr
        

In [16]:
blocks = shift_blocks(blocks); blocks

array(['0', '0', '9', '9', '8', '1', '1', '1', '8', '8', '8', '2', '7',
       '7', '7', '3', '3', '3', '6', '4', '4', '6', '5', '5', '5', '5',
       '6', '6'], dtype='<U21')

In [17]:
positions = np.arange(0, len(blocks))
checksum = np.sum(positions * blocks.astype(int)); checksum

np.int64(1928)

#### Approach 2:

`shift_blocks` isn't efficient for a larger dataset - we can do better

In [18]:
def replace_dots_with_reversed(arr):
    # reverse array and remove dots
    reversed_array = arr[::-1][arr[::-1] != '.']
    # find all indices for dots and replace
    dot_indices = np.where(arr == '.')
    arr[dot_indices] = reversed_array[:len(arr[dot_indices])]
    # num of ints in array should be the same as originally
    return arr[:len(reversed_array)]

In [19]:
test_array = np.array(['1','.','.','2','.','3','4'])

In [20]:
replace_dots_with_reversed(test_array)

array(['1', '4', '3', '2'], dtype='<U1')

In [21]:
replace_dots_with_reversed(blocks)

array(['0', '0', '9', '9', '8', '1', '1', '1', '8', '8', '8', '2', '7',
       '7', '7', '3', '3', '3', '6', '4', '4', '6', '5', '5', '5', '5',
       '6', '6'], dtype='<U21')

In [22]:
data = get_data(day=9, year=2024)

In [23]:
import itertools

def calculate(data):
    # set up data
    counts = np.array(list(data), dtype=int)
    file_values = np.arange(0, len(data)//2 + 1)
    empty_values = np.full(len(data)//2 + 1, '.')
    interleaved = list(itertools.chain(*zip(file_values, empty_values)))
    blocks = np.repeat(interleaved[:len(counts)], counts)

    # run "defragger"
    blocks = replace_dots_with_reversed(blocks)
    
    positions = np.arange(0, len(blocks))
    checksum = np.sum(positions * blocks.astype(int))
    
    return checksum

In [24]:
calculate(data)

np.int64(6370402949053)

## Part Two

The eager amphipod already has a new plan: rather than move individual blocks, he'd like to try compacting the files on his disk by moving whole files instead.

This time, attempt to move whole files to the leftmost span of free space blocks that could fit the file. Attempt to move each file exactly once in order of decreasing file ID number starting with the file with the highest file ID number. If there is no span of free space to the left of a file that is large enough to fit the file, the file does not move.

Start over, now compacting the amphipod's hard drive using this new method instead. What is the resulting filesystem checksum?



In [101]:
class Block:
    b_id: str
    length: int

    def __init__(self, b_id, length):
        self.b_id = str(b_id)
        self.length = int(length)

    def __repr__(self):
        return self.b_id * self.length

In [102]:
class Space:
    length: int

    def __init__(self, length):
        self.length = length

    def shrink_space(self, length):
        "When a file is placed in the space, the space shrinks"
        self.length -= length

    def __repr__(self):
        return "." * self.length

Now use the above to represent our example input:

In [103]:
counts

array([2, 3, 3, 3, 1, 3, 3, 1, 2, 1, 4, 1, 4, 1, 3, 1, 4, 0, 2])

In [104]:
interleaved

array(['0', '.', '1', '.', '2', '.', '3', '.', '4', '.', '5', '.', '6',
       '.', '7', '.', '8', '.', '9', '.'], dtype='<U21')

In [228]:
files = []
for i, j in zip(counts, interleaved):
    if j == '.':
        files.append(Space(i))
    else:
        files.append(Block(j, i))
        
''.join([str(f) for f in files])

'00...111...2...333.44.5555.6666.777.888899'

In [229]:
def recursive_sort(my_files):
    sorted_files = []

    last_file = my_files[-1]
    for i, el in enumerate(my_files):
        print(el)
        if isinstance(el, Space) and isinstance(last_file, Block):
            print(f"Moving {last_file=}")
            if last_file.length <= el.length:
                el.shrink_space(last_file.length)
                print(el)
                my_files.insert(i, last_file)  
                print(''.join([str(f) for f in my_files[:-1]]))
                print('Sort again')
                sorted_files = recursive_sort(my_files[:-1])  

    # if no space was found, put the block back at the end
    sorted_files.append(last_file)  
    
    return sorted_files

In [230]:
sort_1 = recursive_sort(files)

00
...
Moving last_file=99
.
0099.111...2...333.44.5555.6666.777.8888
Sort again
00
99
.
111
...
2
...
333
.
44
.
5555
.
6666
.
777
.
8888

.
Moving last_file=99
111
...
Moving last_file=99
.
0099.11199.2...333.44.5555.6666.777.8888
Sort again
00
99
.
111
99
.
2
...
333
.
44
.
5555
.
6666
.
777
.
8888

.
Moving last_file=99
2
...
Moving last_file=99
.
0099.11199.299.333.44.5555.6666.777.8888
Sort again
00
99
.
111
99
.
2
99
.
333
.
44
.
5555
.
6666
.
777
.
8888

.
Moving last_file=99
333
.
Moving last_file=99
44
.
Moving last_file=99
5555
.
Moving last_file=99
6666
.
Moving last_file=99
777
.
Moving last_file=99
8888

Moving last_file=99
99


In [231]:
''.join([str(f) for f in sort_1])

'99'

In [146]:
''.join([str(f) for f in recursive_sort(sort_1)])

'0099.111...2...333.44.5555.6666.777.8888'

In [147]:
sort_1

[00, 99, ., 111, ..., 2, ..., 333, ., 44, ., 5555, ., 6666, ., 777, ., 8888, ]

Still not right - need to be moving the 2 and 4's.. why does it stop?